In [ ]:
# Problem statement
# 14 controls
# 1 target
# 5 auxilliary 

# b0-13, t, a0-4

# Design 1: (3-4-3)
# 4 3-bit in place control gates (b0-11)->(a0-3)
# 1 4-bit in place control gate taking 2 remaining bits and 2 auxiliary bits (b12-13, a0-1) -> a4
# 3 3-bit in place control gate that takes (a2-4) -> t
